In [1]:
import pandas as pd
complaints = pd.DataFrame([
  {"complaint_id":"CMP-001","customer_id":1002,"category":"Billing","description":"Charged extra for data usage","created_at":"2025/09/25 10:45","status":"Open"},
  {"complaint_id":"CMP-002","customer_id":1004,"category":"Network","description":"Frequent call drops in Delhi","created_at":"2025-09-25 09:30","status":"Open"},
  {"complaint_id":"CMP-003","customer_id":1005,"category":"Recharge","description":"Recharge failed; amount deducted","created_at":"25-09-2025 14:00","status":"Closed"},
  {"complaint_id":"CMP-004","customer_id":1002,"category":"Network","description":"Slow 4G speed at night","created_at":"2025-09-26 20:40","status":"Open"},
  {"complaint_id":"CMP-005","customer_id":1003,"category":"Support","description":"No response to complaint","created_at":"2025-09-26 11:10","status":"Open"}
])
complaints.to_csv("complaints.csv", index=False)
print("✅ complaints.csv saved.")


✅ complaints.csv saved.


In [3]:

from sqlalchemy import create_engine
# Replace with your actual credentials
user = 'root'
password = 'admin'
host = 'localhost'
database = 'test'
# Create SQLAlchemy engine
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")
# Check connection
print("✅ Connected to MySQL successfully!")


✅ Connected to MySQL successfully!


# --- Standardize text ---
customers['region']  = customers['region'].str.title().str.strip()
complaints['status'] = complaints['status'].str.title().str.strip()
complaints['category'] = complaints['category'].str.title().str.strip()

# --- Parse and standardize dates ---
customers['join_date']  = pd.to_datetime(customers['join_date'],  errors='coerce')
complaints['created_at'] = pd.to_datetime(complaints['created_at'], errors='coerce')

# Fill unparseable or missing dates
default_dt = pd.Timestamp('2025-09-25 00:00')
customers['join_date']  = customers['join_date'].fillna(default_dt)
complaints['created_at'] = complaints['created_at'].fillna(default_dt)

# --- Fix missing IDs or text ---
complaints['customer_id'] = complaints['customer_id'].fillna(-1).astype(int)
complaints['description'] = complaints['description'].fillna("No description provided")

# --- Merge ---
merged = customers.merge(complaints, on='customer_id', how='left')

# --- Post-merge fixes ---
merged['complaint_id'] = merged['complaint_id'].fillna("NO-COMPLAINT")
merged['category']     = merged['category'].fillna("No Complaint")
merged['status']       = merged['status'].fillna("Resolved")
merged['created_at']   = merged['created_at'].fillna(default_dt)

# Derived flag
merged['is_open'] = (merged['status'] == 'Open')
print("✅ Data transformed successfully!")
merged.head()
